In [ ]:
!pip install albumentations

In [ ]:
import torch
import albumentations
from albumentations import ( Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, IAAAdditiveGaussianNoise, Transpose, ToGray )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import matplotlib.pyplot as plt

seed = 42

import pandas as pd
import os
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm

import cv2

In [ ]:
class Ranzcr_jpg_train_dataset(Dataset):    
	def __init__(self, files_folder_path, df, num_channels , transfroms = None ):
		self.files_folder_path = files_folder_path
		self.df = df
		self.transforms = transfroms
		self.num_channels = num_channels

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):

		image_id = self.df.StudyInstanceUID.values[idx]
        
		if self.num_channels == 1:
			image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ), 0)

		else:
			image = cv2.imread(os.path.join(self.files_folder_path, image_id + ".jpg" ))
			image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		

		if self.transforms:
			image = self.transforms(image=image)['image']

		labels = self.df[self.df.StudyInstanceUID == image_id].values.tolist()[0][1:-1]
		labels = torch.tensor(labels,dtype= torch.float32) #.view(1,-1)

		return image, labels

In [ ]:
train_path = '../input/ranzcr-clip-catheter-line-classification/train'
train_files = os.listdir(train_path)

train_df = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')
#train_df = pd.read_csv('../input/ranzcr/X_test_small_250_seed_42.csv')

train = train_df.reset_index(drop=True) # reset index on both dataframes

print(train.shape)

num_channel = 3

data_mean = [0.485,0.456,0.406]
data_std = [0.229,0.224,0.225]

#Image augmentation
img_size = 255
print(data_mean , data_std)

In [ ]:

def Show_Xrays(augmentation):
    num_channels = 3
    trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , augmentation )
    trainloader = DataLoader(trainset, batch_size = 4 , num_workers = 0 , shuffle = False)

    fig = plt.figure()
    fig.set_size_inches(15, 15)
    #fig.savefig('test2png.png', dpi=100)

    for batch_i, (data, target) in tqdm(enumerate(trainloader)):
        #print(data.shape)
        if batch_i == 1:
            break
        for i in range(data.shape[0]):
            ax = plt.subplot(2 ,2, i+1)
            plt.tight_layout()
            ax.axis('off')
            plt.imshow(data[i])

# Baseline

In [ ]:
train_augs = albumentations.Compose([ albumentations.Resize(height=img_size, width=img_size, p=1.0), albumentations.Normalize(mean= data_mean,std= data_std ,),])

Show_Xrays(train_augs)

In [ ]:
train_augs = albumentations.Compose([   albumentations.Resize(img_size, img_size),    ToTensorV2()     ])

num_channels = 3
trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , train_augs )
trainloader = DataLoader(trainset, batch_size = 64 , num_workers = 1 , shuffle = True)

In [ ]:
for batch_i, (data, target) in tqdm(enumerate(trainloader)):
        print(data.shape)
        if batch_i == 1:
            break

In [ ]:
data[0,:, 1,1]

https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/Basics/pytorch_std_mean.py
https://www.youtube.com/watch?v=y6IEcEBRZks

In [ ]:
train_augs = albumentations.Compose([   albumentations.Resize(img_size, img_size),    ToTensorV2()     ])

num_channels = 3
trainset = Ranzcr_jpg_train_dataset(train_path,  train, num_channels , train_augs )
trainloader = DataLoader(trainset, batch_size = 64 , num_workers = 1 , shuffle = True)

def get_mean_std(loader):
    # var[X] = E[X**2] - E[X]**2
    channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0
    for data, _ in tqdm(loader):
        channels_sum += torch.Tensor.float(data ).mean(dim=[0, 2, 3])
        print(num_batches +1  , channels_sum/num_batches+1)
        channels_sqrd_sum += torch.Tensor.float(data ** 2).mean(dim=[0, 2, 3])
        print(num_batches +1  , channels_sqrd_sum/num_batches+1)
        num_batches += 1
    
    mean = channels_sum / num_batches
    std = (channels_sqrd_sum / num_batches - mean ** 2) ** 0.5
    std_not_minus = (channels_sqrd_sum / num_batches ) ** 0.5
    
    return mean, std , std_not_minus

In [ ]:
mean, std , mean_std = get_mean_std(trainloader)

In [ ]:
mean, std , mean_std = get_mean_std(trainloader)

In [ ]:
mean = [124.3329, 124.3329, 124.3329]
std = [103.0483, 103.0483, 103.0483]

std = (((std - mean) ** 2) * (1 / num_batches))  ** 0.5

In [ ]:
print(mean, std)

In [ ]:
mean = [123.0365, 123.0365, 123.0365]
#mean = mean*(1/255)
std = [0.5,0.5,0.5]

data_mean = [0.485,0.456,0.406]
data_std = [0.229,0.224,0.225
            
mean = [130.4711, 130.4711, 130.4711]
mean = mean*(1/255)
print(mean)
std = [108.1780, 108.1780, 108.1780]
std = std*(1/255)
print(std)
            
            
57 tensor([126.3339, 126.3339, 126.3339])
57 tensor([104.8026, 104.8026, 104.8026])

92 tensor([125.4034, 125.4034, 125.4034])
92 tensor([104.0455, 104.0455, 104.0455])

            
114 tensor([125.3113, 125.3113, 125.3113])
114 tensor([103.8559, 103.8559, 103.8559])

            

In [ ]:
123.0365/255

In [ ]:
train_augs = albumentations.Compose([ albumentations.Resize(height=img_size, width=img_size, p=1.0), 
                                      albumentations.Normalize(mean= mean, std= std ),])

Show_Xrays(train_augs)

In [ ]:
R_channel = 0
G_channel = 0
B_channel = 0

pathDir = '../input/ranzcr-clip-catheter-line-classification/test/*.jpg'

from glob import glob
img_list = glob(pathDir)
print(len(img_list))

In [ ]:
R_total , G_total ,B_total = 0, 0 , 0

total_pixel = 0
for idx in range(len(img_list)):
    filename = img_list[idx]
    img = plt.imread(filename)

    total_pixel = total_pixel + img.shape[0] * img.shape[1]

    R_total += np.sum((img[:, :, 0] - R_mean) ** 2)
    G_total = G_total + np.sum((img[:, :, 1] - G_mean) ** 2)
    B_total = B_total + np.sum((img[:, :, 2] - B_mean) ** 2)


In [ ]:

R_std = sqrt(R_total / total_count)
G_std = sqrt(G_total / total_count)
B_std = sqrt(B_total / total_count)

In [ ]:
traindata = datasets.ImageFolder('../input/ranzcr-clip-catheter-line-classification/test/', transforms.ToTensor())

In [ ]:
image_means = torch.stack([t.mean(1).mean(1) for t, c in traindata])
image_means.mean(0)
